## Hierarchical LSTM For Position Estimation

### Load libs

In [1]:
%matplotlib inline
# %pylab osx
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipyd
plt.style.use('ggplot')


### Load data and create database

In [2]:
from Dataset import Train_And_Test_Data
Data=Train_And_Test_Data()
Data.load_data()
Data.min_and_max()
[X_train,Arm_train,y_train,X_test,Arm_test,y_test]=Data.Dataset()

X_train : Train spiking data shape:  (1, 23209, 62)
Arm_train : Train maze labels shape:  (1, 23209, 4)
y_train : train position data shape:  (1, 23209, 4)
X_test : Test spiking data shape:  (1, 4095, 62)
Arm_test : Test maze labels shape:  (1, 4095, 4)
y_test : Test position data shape:  (1, 4095, 4)


### Create Position Estimator with penalized loss

In [3]:
from decoder import Full_Model_LSTMDecoder
Decoder=Full_Model_LSTMDecoder(verbose=0,Topology=2)
Decoder.create_model(X_train,y_train)


Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 23209, 62)    0                                            
__________________________________________________________________________________________________
MazeSeq (LSTM)                  (None, 23209, 4)     1072        Input[0][0]                      
__________________________________________________________________________________________________
ConcatFeature (Concatenate)     (None, 23209, 66)    0           MazeSeq[0][0]                    
                                                                 Input[0][0]                      
__________________________________________________________________________________________________
P_Features (LSTM)               (None, 23209, 2)     552         ConcatFeature[0][0]              
__________

### Train The Model

In [4]:
Decoder.fit(X_train,y_train,Arm_train)

### Estimate the test path

In [5]:
np.max(y_predict[:,0])

NameError: name 'y_predict' is not defined

In [ ]:
[y_predict,Maze]=Decoder.predict(X_test)


y_predict[:,0]=np.multiply(y_predict[:,0],Data.MaxX-Data.MinX)+Data.MinX
y_predict[:,1]=np.multiply(y_predict[:,1],Data.MaxY-Data.MinY)+Data.MinY
y_predict[:,2]=np.multiply(y_predict[:,2],Data.VMaxX-Data.VMinX)+Data.VMinX
y_predict[:,3]=np.multiply(y_predict[:,3],Data.VMaxY-Data.VMinY)+Data.VMinY



### Get measures

### RMSE

In [ ]:
from metrics import get_RMSE
RMSE=get_RMSE(y_test[0],y_predict)
RMSE

### R2

In [ ]:
from metrics import get_R2
R2=get_RMSE(y_test[0],y_predict)
R2

### Show the result

In [ ]:
plt.figure(num=None, figsize=(12, 6))
plt1=plt.plot(y_predict[:,1],'b',label="EstimatedTestPath-Y")
plt.plot(y_test[0,:,1],'r',label="TestPath-Y")
plt.legend(loc=4)
plt.title('Estimate position Y')
#plt.savefig('EstimatedTestPath-Y2.png',format='png', dpi=1000,transparent=False)

plt.figure(num=None, figsize=(12, 6))
plt1=plt.plot(y_predict[:,0],'b',label="EstimatedTestPath-X")
plt.plot(y_test[0,:,0],'r',label="TestPath-X")
plt.legend(loc=4)
plt.title('Estimate position X ')
#plt.savefig('EstimatedTestPath-X2.png',format='png', dpi=1000,transparent=False)

plt.figure(num=None, figsize=(12, 6))
plt.plot(np.argmax(Maze[0],axis=-1),'b',label="EstimatedMaze")
plt.plot(np.argmax(Arm_test[0],axis=-1),'r',label="TestMaze")
plt.legend(loc=4)
plt.title('Maze')

#plt.savefig('EstimatedTestPath-X2.png',format='png', dpi=1000,transparent=False)
plt.figure(num=None, figsize=(12, 6))
plt1=plt.plot(y_predict[:,2],'b',label="EstimatedVelocity-X")
plt.plot(y_test[0,:,2],'r',label="TestVelocity-X")
plt.legend(loc=4)
plt.title('Estimate Velocity X')
#
plt.figure(num=None, figsize=(12, 6))
plt1=plt.plot(y_predict[:,3],'b',label="EstimatedVelocity-Y")
plt.plot(y_test[0,:,3],'r',label="TestVelocity-Y")
plt.legend(loc=4)
plt.title('Estimate Velocity Y ')